# Get BBC Dataset

In [ ]:
from google.colab import drive
drive.mount("./gdrive")

In [ ]:
cd gdrive/My\ Drive/Colab\ Notebooks/Datasets/Text\ Summarization

In [ ]:
!pip show tensorflow

In [ ]:
import pickle 
with open('myfile.pkl','rb') as fp:
  heading, article = pickle.load(fp)

In [ ]:
leny=[]
for i in range(19000):
    if len(article[i]) < 351:
        leny.append(i)

In [ ]:
leny = leny[0:10]
short_article = []
short_heading = []
for i in leny:
    short_article.append(article[i])
    short_heading.append(heading[i])

In [ ]:
short_article[3]

**Data from BBC News Dataset Loaded. The variables are *heading* and *article***

# Preprocess the Data , remove the \n\n and \xa0 , lower case everything  (stemming/lemmatization advantageous? Try it out later)

## I think I should keep the comma's in for now , maybe the Attention mechanism later would find words near a comma a good place to give attention to

In [ ]:
short_heading = [h.lower() for h in short_heading]
short_article = [a.lower() for a in short_article]

In [ ]:
short_article = [a.replace('\n','').replace('\xa0',' ').replace('\r','').replace('\t','') for a in short_article]

In [ ]:
import re

REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")

def preprocess(reviews):
  reviews = [REPLACE_NO_SPACE.sub("",line.lower()) for line in reviews]
  reviews = [REPLACE_WITH_SPACE.sub(" ",line) for line in reviews]
  
  return reviews

In [ ]:
short_articles = preprocess(short_article)
short_headings = preprocess(short_heading)

# Toeknize the data , Make Word Embeddings from GloVe

## Mainly what I have to do is make a matrix of size (vocab_size, 100) which I can initialize as my weight matrix. Be careful, for now I am assuming no new words can appear in a text other than the ones I have seen before , so later if this works improve it to take in other words too

In [ ]:
word_dimensions = 100
import pandas as pd
df = pd.read_csv('glove.6B.100d.txt', sep=" ", quoting=3, header=None, index_col=0)

In [ ]:
glove = {key: val.values for key, val in df.T.items()}

In [ ]:
import nltk
from collections import Counter
import numpy as np

In [ ]:
words = []
tokenizer = nltk.tokenize.TreebankWordTokenizer()
for a in short_articles:
  tokenized_text = tokenizer.tokenize(a)
  for tokens in tokenized_text:
    words.append(tokens)
    
for h in short_headings:
  tokenized_text = tokenizer.tokenize(h)
  for tokens in tokenized_text:
    words.append(tokens)

In [ ]:
import nltk
nltk.download('wordnet')

In [ ]:
wordcounts = Counter(words).most_common(500)
vocab_list = np.array([word for word,_ in wordcounts])
dictionary_word_code = {word:code for code,word in enumerate(vocab_list)}

In [ ]:
embedding_list = []
i = 0
while i != vocab_list.shape[0] :
  try:
    embedding_list.append(glove[vocab_list[i]])
    i = i+1
  except KeyError as e:
    embedding_list.append(np.random.normal(0,0.1,[word_dimensions]))
    i = i+1
    
  if i%100 == 0:
    print(i)

In [ ]:
embedding_matrix = np.vstack(embedding_list)

In [ ]:
embedding_matrix.shape

**Embedding Matrix is created and in the variable *embedding_matrix***

# Making a Bi-Directional RNN with Attention , no dropout layer

## For now making a 3 Layer LSTM deep network (could cause vanishing gradient problem) and not using the EOS thing, will make the summary a fixed number of words , mostly wont be coherent but lets see if any output comes at all first.

In [ ]:
import tensorflow as tf

#### 'The' also has a code of 0 and you are gonna zero pad things so could lead to problems

#### Also you are using too much RAM afterwards change variable names so you overwriting them and not keep making new ones
Ok so for saving RAM I am thinking I will make a pickle file for all the important stuff and load it in so all the useless variables dont take up space

In [ ]:
tokenized_articles = []
tokenized_headings = []
for a in short_articles:
  tokens = tokenizer.tokenize(a)
  tokenized_articles.append(tokens)
for h in short_headings:
  tokens = tokenizer.tokenize(h)
  tokenized_headings.append(tokens)
  
max_length_article = max([len(a) for a in tokenized_articles])
max_length_heading = max([len(h) for h in tokenized_headings])

numerical_articles = []
numerical_headings = []
for a in tokenized_articles:
    list1 = [dictionary_word_code[b] for b in a]
    numerical_articles.append(list1)
for h in tokenized_headings:
  list1 = [dictionary_word_code[b] for b in h]
  numerical_headings.append(list1)
  
padded_numerical_articles = []
padded_numerical_headings = []
for a in numerical_articles:
  zeros = [0]*(max_length_article - len(a))
  list1 = a + zeros
  padded_numerical_articles.append(list1)
for h in numerical_headings:
  zeros = [0]*(max_length_heading - len(h))
  list1 = h + zeros
  padded_numerical_headings.append(list1)
  
with open("input_data.pickle", "wb") as f:
    pickle.dump((embedding_matrix,padded_numerical_articles,padded_numerical_headings,vocab_list), f)

# Too much RAM was used so start from here

In [1]:
import pickle 
with open("input_data.pickle","rb") as f:
  embedding_matrix , articles , headings, vocab_list = pickle.load(f)

In [20]:
headings[1][0:13]

[283, 284, 285, 3, 286, 3, 0, 287, 24, 45, 16, 0, 0]

In [ ]:
headings[4][0:20]

In [ ]:
embedding_matrix[238]

In [ ]:
# import numpy as np
# def next_batch(articles,headings,batch_size,num_time_steps,num_output_steps):                 #850     #84
  
#   snipped_articles = []
#   snipped_headings = []
  
#   for a in articles:
#     part_article = a[0:num_time_steps]
#     snipped_articles.append(part_article)
    
#   random = np.random.randint(0,len(articles) - batch_size -1)
#   batch_x = np.array(snipped_articles[random:random+batch_size]).reshape([batch_size,num_time_steps,1])
#   onehotlistarticles = []
# #  for article in batch_x:
# #    for num in article:
# #      zeros = np.array([0]*len(vocab_list))
# #      zeros[num] = 1
# #      onehotlistarticles.append(zeros)
# #  onehotlistarticles = np.array(onehotlistarticles).reshape([batch_size,num_time_steps,-1])
  
#   for h in headings:
#     part_heading = h[0:num_output_steps]
#     snipped_headings.append(part_heading)
  
#   batch_y = np.array(snipped_headings[random:random+batch_size]).reshape([batch_size,num_output_steps,1])
#   onehotlistheadings = []
#   for heading in batch_y:
#     for num in heading:
#       zeros = np.array([0]*len(vocab_list))
#       zeros[num] = 1
#       onehotlistheadings.append(zeros)
#   onehotlistheadings = np.array(onehotlistheadings).reshape([batch_size,num_output_steps,-1])
  
#   return onehotlistarticles , onehotlistheadings

In [65]:
import numpy as np
def next_batch(articles,headings,batch_size,num_time_steps,num_output_steps):                 #850     #84
  
  snipped_articles = []
  snipped_headings = []
  
  for a in articles:
    part_article = a[0:num_time_steps]
    snipped_articles.append(part_article)
    
  random = np.random.randint(0,len(articles) - batch_size -1)
  batch_x = np.array(snipped_articles[random:random+batch_size]).reshape([batch_size,num_time_steps,1])
  #batch_x = np.array([snipped_articles[2],snipped_articles[4]]).reshape([batch_size,num_time_steps,1])
  embeddedarticles = []
  for article in batch_x:
    for num in article:
        value = embedding_matrix[num]
        embeddedarticles.append(value)
  
  embeddedarticles = np.array(embeddedarticles).reshape([batch_size,num_time_steps,-1])
  
  for h in headings:
    part_heading = h[0:num_output_steps]
    snipped_headings.append(part_heading)
  
  batch_y = np.array(snipped_headings[random:random+batch_size]).reshape([batch_size,num_output_steps,1])
  #batch_y = np.array([snipped_headings[2],snipped_headings[4]]).reshape([batch_size,num_output_steps,1])
  onehotlistheadings = []
  for heading in batch_y:
    for num in heading:
      zeros = np.array([0]*len(vocab_list))
      zeros[num] = 1
      onehotlistheadings.append(zeros)
  onehotlistheadings = np.array(onehotlistheadings).reshape([batch_size,num_output_steps,-1])
  
  return embeddedarticles , onehotlistheadings

In [63]:
onehotencodedarticle , onehotencodedheading = next_batch(articles,headings,5,55,13)

In [64]:
onehotencodedarticle.shape

(5, 55, 100)

In [ ]:
type(embedding_matrix.shape[1])

# TensorFlow Graph 

## Encoder 

In [66]:
def length(sequence):
  used = tf.reduce_max(sequence, 2)
  length = tf.reduce_sum(used, 1)
  length = tf.cast(length, tf.int32)
  return length

**See Imext Notebook for explaination of tf.reduce_max and how axis collapses and stuff**

In [67]:
import tensorflow as tf
#import numpy as np

In [68]:
num_time_steps = 55            #850
num_output_steps = 13            #84
dimensions_rnn_cell = 50         #100
batch_size = 7

In [69]:
tf.reset_default_graph()

In [70]:
X = tf.placeholder(tf.float32,shape = [None,num_time_steps,embedding_matrix.shape[1]])

** Right now too tired, not initializing with Embedding Matrix as its a pain , have no clue what the fuck the function wants from me, come back later here if the performance is not so good , see the documentation for initializer in LSTMCell**

In [71]:
lstm_fw_cell = [tf.nn.rnn_cell.LSTMCell(dimensions_rnn_cell),tf.nn.rnn_cell.LSTMCell(dimensions_rnn_cell)] #,tf.nn.rnn_cell.LSTMCell(dimensions_rnn_cell)]
lstm_bw_cell = [tf.nn.rnn_cell.LSTMCell(dimensions_rnn_cell),tf.nn.rnn_cell.LSTMCell(dimensions_rnn_cell)] #,tf.nn.rnn_cell.LSTMCell(dimensions_rnn_cell)]

outputs, output_state_fw, output_state_bw = tf.contrib.rnn.stack_bidirectional_dynamic_rnn(lstm_fw_cell, lstm_bw_cell, X, dtype='float32', sequence_length = length(X))

** Cool so session crashes after using all the RAM**

** For testing if this is working or not **

In [ ]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
  sess.run(init)
  batch_x, v = next_batch(articles,headings,batch_size,100,10)
  combined_bi_rnn_output = sess.run(outputs,feed_dict = {X:batch_x})

In [ ]:
combined_bi_rnn_output

In [ ]:
## Attention Mechanism : For now, wont be taking input from the final output of the network and lets see how it performs, afterwards take as input from the output of the final stage too 

## I think thats possible because the final outputs are also placeholders right? So that can be input to your 'neural_network' function

### Define a Neural Network basically 200 : 100 : 84

In [72]:
combined_BiRNN_outputs = tf.placeholder(tf.float32,shape = [None,num_time_steps,2*dimensions_rnn_cell])

In [73]:
num_hidden_neurons = 30
def neural_network(combined_BiRnn_output):
    
    Wx1 = tf.Variable(tf.random_normal([batch_size,2*dimensions_rnn_cell,num_hidden_neurons],1,1/100))
    Wx2 = tf.Variable(tf.random_normal([batch_size,num_hidden_neurons,num_output_steps],1,1/30))
    b1 = tf.Variable(tf.constant(0.0,shape = [num_hidden_neurons]))
    b2 = tf.Variable(tf.constant(0.0,shape = [num_output_steps]))
    first_out = tf.nn.tanh(tf.add(tf.matmul(combined_BiRnn_output,Wx1),b1))
    e = tf.nn.relu(tf.add(tf.matmul(first_out,Wx2),b2))
        
    return e

In [74]:
def softmax(e_values):
    j = tf.exp(e_values)
    totals = tf.reduce_sum(j,1)
    j = tf.unstack(j,axis = 0)
    totals = tf.unstack(totals, axis = 0)
    for i in range(batch_size):
        j[i] = j[i]/totals[i]
    j = tf.stack(j,axis = 0)
    return j

In [75]:
e_vals = neural_network(combined_BiRNN_outputs)
alpha_values = softmax(e_vals)

** To test whether alpha values are coming or not **

In [ ]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    batch_x, _ = next_batch(articles,headings,batch_size,num_time_steps,num_output_steps)
    outs = sess.run(outputs,feed_dict = {X: batch_x})
    
    alpha = sess.run(alpha_values,feed_dict = {combined_BiRNN_outputs:outs})

In [ ]:
alpha[0]

** Weighted Average **

In [76]:
def weighted_average(outputs,alphas):
    
    alphas = tf.transpose(alphas,perm = [0,2,1])
    context_vectors = tf.matmul(alphas,outputs)
    
    return context_vectors

In [77]:
context_vectors = weighted_average(combined_BiRNN_outputs,alpha_values)

### Another idea you could do is instead of one hot encoding the input words use the embedding matrix you created, though wont be as useful if you could load up the embedding matrix as the weight vector it may be better than normal one hot encoding

### Or use a LSTM cell as the first layer and top it off with 2 BiRnn layers or something if LSTM cell supports initialzing 


In [78]:
context = tf.placeholder(tf.float32,shape = [batch_size,num_output_steps,2*dimensions_rnn_cell])

In [79]:
cell_factory = tf.nn.rnn_cell.LSTMCell(dimensions_rnn_cell,activation=tf.nn.relu)
initial_state = cell_factory.zero_state(batch_size, dtype=tf.float32)
decoder, _ = tf.nn.dynamic_rnn(cell_factory,context,initial_state=initial_state)

In [80]:
def output_neural_network(decoder_rnn):
    
    Wx1 = tf.Variable(tf.random_normal([batch_size,dimensions_rnn_cell,len(vocab_list)],0.5,1/dimensions_rnn_cell))
    b1 = tf.Variable(tf.constant(0.0,shape = [len(vocab_list)]))
    first_out = tf.nn.relu((tf.add(tf.matmul(decoder_rnn,Wx1),b1)))
        
    return first_out

In [81]:
probability_distribution = output_neural_network(decoder)

In [82]:
y_true = tf.placeholder(tf.float32,shape = [None,num_output_steps,len(vocab_list)])

In [83]:
c_e = tf.nn.softmax_cross_entropy_with_logits_v2(logits = probability_distribution,labels = y_true)
c_e_s = tf.reduce_mean(c_e)
mse = tf.reduce_mean(tf.square(probability_distribution-y_true))
optimizer = tf.train.AdamOptimizer(learning_rate = 0.002)
train = optimizer.minimize(c_e_s)

In [84]:
saver = tf.train.Saver()

In [48]:
batch_x , batch_y = next_batch(articles,headings,batch_size,num_time_steps,num_output_steps)

In [85]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
  sess.run(init)
  #batch_x , batch_y = next_batch(articles,headings,batch_size,num_time_steps,num_output_steps)
  for i in range(500):
    batch_x , batch_y = next_batch(articles,headings,batch_size,num_time_steps,num_output_steps)
    outs = sess.run(outputs, feed_dict = {X:batch_x})
    con = sess.run(context_vectors, feed_dict = {combined_BiRNN_outputs:outs})
    sess.run(train, feed_dict = {context:con,y_true:batch_y})
    
    if i%100 == 0:
      print('Cross entropy at Step {0} : '.format(i))
      print(sess.run(c_e_s,feed_dict = {context:con,y_true:batch_y}))
      print("\n")
      
  saver.save(sess,'./imext_summarizer_1.ckpt')
      
    #batch_x , batch_y = next_batch(articles,headings,1,100,10)
    #outs = sess.run(outputs, feed_dict = {X:batch_x})
    #con = sess.run(context_vectors, feed_dict = {combined_BiRNN_outputs:outs})
    #final_output = sess.run(probability_distribution, feed_dict = {context:con})
      

Cross entropy at Step 0 : 
5.734525


Cross entropy at Step 100 : 
1.5827332


Cross entropy at Step 200 : 
1.0446652


Cross entropy at Step 300 : 
0.5008614


Cross entropy at Step 400 : 
0.24542199




In [86]:
batch_x , batch_y = next_batch(articles,headings,batch_size,num_time_steps,num_output_steps)
with tf.Session() as sess:
    saver.restore(sess,'./imext_summarizer_1.ckpt')
    
    outs = sess.run(outputs, feed_dict = {X:batch_x})
    con = sess.run(context_vectors, feed_dict = {combined_BiRNN_outputs:outs})
    final_output = sess.run(probability_distribution, feed_dict = {context:con})


INFO:tensorflow:Restoring parameters from ./imext_summarizer_1.ckpt


In [88]:
final_output.shape

(7, 13, 315)

In [91]:
numerical_words = np.argmax(final_output[1],axis = 1)

In [58]:
numerical_words

array([18, 19,  1,  6, 48, 49,  3, 20,  1, 21,  2, 22, 50])

In [92]:
head = []
for i in numerical_words:
    head.append(vocab_list[i])
head

['survivor',
 'upset',
 'use',
 'of',
 'eye',
 'of',
 'the',
 'tiger',
 'with',
 'kim',
 'davis',
 'the',
 'the']

In [56]:
arti = []
for i in headings[4]:
    arti.append(vocab_list[i])
arti

['dark',
 'and',
 'full',
 'bodied',
 'does',
 'it',
 'for',
 'me',
 'every',
 'time',
 'the',
 'the',
 'the']

In [ ]:
final_output[0][0][0:20]